## Inferencia y evaluación

### 0 - Configuracion Sagemaker

In [ ]:
import boto3
import pandas as pd
import os
from io import StringIO

import sagemaker
from sagemaker.image_uris import retrieve
from sagemaker.serializers import CSVSerializer

In [ ]:
# identificación de región
region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

# indetificación de rol
role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))

# sesión de sagemaker
sagemaker_session = sagemaker.Session()

### 1 - Inferencia

**Cargamos datos locales (real-time request)**

In [ ]:
data_dir_test = "./data/pd/test"
df_test = pd.read_csv(os.path.join(data_dir_test, 'test.csv'), header=None)

In [ ]:
# separamos variables y objetivo
df_y_test = df_test[0]
df_x_test = df_test.loc[:, df_test.columns != 0]

In [ ]:
# min 1 max 1000
num_predicciones = 50
records_to_predict = df_x_test[:num_predicciones]

In [ ]:
# serializamos los datos
# por defecto sagemaker espera un csv divido en comas
csv_file = StringIO()
records_to_predict.to_csv(csv_file, sep=",", header=False, index=False)
payload_as_csv = csv_file.getvalue()

**Inferencia**

In [ ]:
from sagemaker.predictor import Predictor

In [ ]:
# instanciamos la clase predictor
endpoint_name = "mi-primer-endpoint-serverless"
predictor = Predictor(endpoint_name=endpoint_name,
                      sagemaker_session=sagemaker.Session(),
                      serializer=CSVSerializer())

In [ ]:
# realizamos la predicción
predicc_res = predictor.predict(payload_as_csv).decode('utf-8')

### 2 - Evaluación del modelo

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# formateamos los resultados
vals = predicc_res.split(',')
predicciones = [round(float(num)) for num in vals]

In [ ]:
# evaluación del accuracy
accuracy_score(df_y_test[:num_predicciones], predicciones)

In [ ]:
# calculamos la matriz de confusión
cm = confusion_matrix(df_y_test[:num_predicciones], predicciones)

In [ ]:
# visualizamos la matriz de confusión
sns.heatmap(cm,
            annot=True,
            fmt='g',
            xticklabels=['Neg','Pos'],
            yticklabels=['Neg','Pos'])
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix',fontsize=17)
plt.show()

### 3 - Borrar endpoint y modelos

In [ ]:
# sdk de aws
sagemaker_bt3 = boto3.client("sagemaker", region_name=region)

**Eliminamos los endpoints**

In [ ]:
endpoint_list = sagemaker_bt3.list_endpoints()
endpoints = [endpoint['EndpointName'] for endpoint in endpoint_list['Endpoints']]
print(f"Endpoints a borrar: {endpoints}")

In [ ]:
for endpoint in endpoints:
    print(f"Borrando endpoint: {endpoint}")
    sagemaker_bt3.delete_endpoint(EndpointName=endpoint)

**Eliminamos la configuración de endpoints**

In [ ]:
endpoints_config_res = sagemaker_bt3.list_endpoint_configs()
endpoints_config = [endpoint['EndpointConfigName'] for endpoint in endpoints_config_res['EndpointConfigs']]
print(f"Configuracion de endpoint a borrar: {endpoints_config}")

In [ ]:
for endpoint_conf in endpoints_config:
    print(f"Borrando configuracion de endpoint: {endpoint_conf}")
    sagemaker_bt3.delete_endpoint_config(EndpointConfigName=endpoint_conf)

**Eliminamos los modelos**

In [ ]:
models_list = sagemaker_bt3.list_models()
models = [model['ModelName'] for model in models_list['Models']]
print(f"Modelos a borrar: {models}")

In [ ]:
for model in models:
    print(f"Borrando modelo: {model}")
    sagemaker_bt3.delete_model(ModelName=model)